In [29]:
import numpy as np
import pandas as pd
import sys
import re
from nerus import load_nerus
from collections import Counter
from scipy.stats.distributions import chi2
import scipy.stats as st
import pymorphy2
from nltk.tag import hmm

In [3]:
docs = load_nerus('./nerus_lenta.conllu.gz')

## Препроцессинг

Напишем функцию разбиения датасета на токены. При этом чтобы одно и то же слово в разных падежах/формах не образовывало несколько состояний, приведём слова в начальную (нормальную) форму.

Нам нужно научиться определять, это именованная сущность или нет. В датасете именованные сущности идут с тэгом 'PROPN'. Для неименованных сущностей сделаем тэг 'NOT PROPN'.

In [16]:
morph = pymorphy2.MorphAnalyzer()

def words_getter(rus_re, mb_count, dataset):
    tokens = []
    megabyte_size = 1024 ** 2
    i = 0
    while sys.getsizeof(tokens) < mb_count * megabyte_size:
        i += 1
        for sent in next(dataset).sents:
            for token in sent.tokens:
                word = morph.parse(token.text)[0].normal_form
                pos = token.pos
                if token.pos == 'PROPN':
                    tokens.append((word, pos))
                else:
                    tokens.append((word, 'NOT PROPN'))
                
    return tokens

In [17]:
tokens = words_getter(russian, 10, docs)

In [18]:
print(len(tokens))
tokens[:10]

1223101


[('депутат', 'NOT PROPN'),
 ('верховный', 'NOT PROPN'),
 ('рада', 'PROPN'),
 ('украина', 'PROPN'),
 ('антон', 'PROPN'),
 ('геращенко', 'PROPN'),
 ('в', 'NOT PROPN'),
 ('разговор', 'NOT PROPN'),
 ('с', 'NOT PROPN'),
 ('рбк', 'PROPN')]

In [27]:
vocab = {word: pos for word, pos in tokens}
len(vocab), list(vocab)[:10]

(45091,
 ['депутат',
  'верховный',
  'рада',
  'украина',
  'антон',
  'геращенко',
  'в',
  'разговор',
  'с',
  'рбк'])

In [24]:
def ngrams_and_prefix_counts(tokens, n_max):
    # словарь n-грамм и их частот
    ngrams_counts = {}
    # словарь n-граммных префиксов и их частот
    prefix_counts = {}
    
    n = len(tokens)
    for i in range(n_max):
        ngrams_counts[i + 1] = Counter([tuple(tokens[j : j + i + 1]) for j in range(n - i)])
        prefix_counts[i + 1] = Counter([tuple(tokens[j : j + i] + ['*']) for j in range(n - i)])

    return ngrams_counts, prefix_counts

def unigram_probas(ngram_counts):
    """
    gets probabilities of unigrams in text
    """
    p1 = {}
    n = sum(ngram_counts[1].values())
    for w in ngram_counts[1]:
        p1[w] = ngram_counts[1][w] / n
    return p1


def bigram_probas(ngram_counts, prefix_counts):
    """
    gets probabilities of bigrams in text
    """
    p2 = {}
    for w in ngram_counts[2]:
        pre_w = tuple([w[0]] + ['*'])
        p2[u'{1}|{0}'.format(*w)] = ngram_counts[2][w] / prefix_counts[2][pre_w]
    return p2


def trigram_probas(ngram_counts, prefix_counts):
    """
    gets probabilities of trigrams in text
    """
    p3 = {}
    for w in ngram_counts[3]:
        pre_w = w[:2] + tuple(['*'])
        p3[u'{2}|{1},{0}'.format(*w)] = ngram_counts[3][w] / prefix_counts[3][pre_w]
    return p3

def chi2_statistic(p2, p3, tokens):
    stat2 = []
    stat3 = []
    n = len(tokens)
    for i in range(n - 2):
        w = tokens[i : i + 3]
        ngram3 = '{2}|{1},{0}'.format(*w)
        ngram2 = '{1}|{0}'.format(*w)

        stat2.append(np.log(p2[ngram2]))
        stat3.append(np.log(p3[ngram3]))
    return - 2 * np.sum(stat2) + 2 * np.sum(stat3)

def chi2_statistic(p_small, p_big, tokens, model_type: str):
    """
    gets chi2 stat for the hypothesis that we can reduce
    the bigram model to the monogram one
    """
    stat_small = []
    stat_big = []
    n = len(tokens)
    for i in range(n - 2):
        if model_type == "3_to_2":
            w = tokens[i : i + 3]
            ngram_big = '{2}|{1},{0}'.format(*w)
            ngram_small = '{1}|{0}'.format(*w)
        elif model_type == "2_to_1":
            w = tokens[i : i + 2]
            ngram_big = '{1}|{0}'.format(*w)
            ngram_small = '{0}'.format(*w)
        else:
            raise ValueError("unrecognized model type")

        try:
            stat_small.append(np.log(p_small[ngram_small]))
        except KeyError:
            stat_small.append(np.log(1e-20))
        try:
            stat_big.append(np.log(p_big[ngram_big]))
        except KeyError:
            stat_big.append(np.log(1e-20))
    return - 2 * np.sum(stat_small) + 2 * np.sum(stat_big)

## Оптимальная длина Марковской цепочки

Воспользуемся кодом с семинара, чтобы проверить, есть ли смысл добавлять цепочки длины 3 и 2.

In [21]:
ngram_counts, prefix_counts = ngrams_and_prefix_counts(tokens, 3)
p1 = unigram_probas(ngram_counts)
p2 = bigram_probas(ngram_counts, prefix_counts)
p3 = trigram_probas(ngram_counts, prefix_counts)

In [26]:
m = len(p3)
print(f'p-value 32 = {1 - st.distributions.chi2(m * ((m - 1) ** 2) - 1).cdf(chi2_statistic(p2, p3, tokens, "3_to_2"))}')
m = len(p2)
print(f'p-value 21 = {1 - st.distributions.chi2(m * ((m - 1) ** 2) - 1).cdf(chi2_statistic(p1, p2, tokens, "2_to_1"))}')

p-value 32 = 1.0
p-value 21 = 1.0


Это означает, что нам достаточно взять модель первого порядка.

## Обучение

В HiddenMarkovModelTrainer первым аргументом подаём список скрытых состояний. Их всего 2: именованные и неименованные сущности.

In [30]:
trainer = hmm.HiddenMarkovModelTrainer(['PROPN', 'NOT PROPN'], vocab)

In [31]:
tagger = trainer.train_supervised([tokens])

Вот мы обучили модель. 

Посмотрим теперь на матрицу переходов между скрытыми состояниями.

In [34]:
trans_matr = pd.DataFrame(
    data=np.array([
        [tagger._transitions['PROPN'].prob('PROPN'), tagger._transitions['PROPN'].prob('NOT PROPN')],
        [tagger._transitions['NOT PROPN'].prob('PROPN'), tagger._transitions['NOT PROPN'].prob('NOT PROPN')]
    ]),
    columns=['PROPN', 'NOT PROPN'],
    index=['PROPN', 'NOT PROPN'])
trans_matr

,PROPN,NOT PROPN
PROPN,0.217889,0.782111
NOT PROPN,0.062029,0.937971


Неименованных сущностей заметно больше, чем именованных, поэтому логично, что после слова скорее всего будет следовать неименованная сущность. Именно из-за этого значения в колонке 'NOT PROPN' больше.

А теперь посмотрим на матрицу выходных вероятностей.

In [35]:
out_matr = pd.DataFrame(
    data=np.array([
        [tagger._outputs['PROPN'].prob(c) for c in vocab],
        [tagger._outputs['NOT PROPN'].prob(c) for c in vocab]
    ]),
    index=['PROPN', 'NOT PROPN'],
    columns=vocab)
out_matr

,депутат,верховный,рада,украина,антон,геращенко,в,разговор,с,рбк,...,изрядно,бриджер-титон,wkmg,чабон,chubon,аптейн,uptain,поскакать,jackson,guide
PROPN,0.000000,0.000000,0.001213,1.795808e-02,0.00089,0.000345,0.000245,0.000000,0.000456,0.001090,...,0.000000e+00,0.000011,0.000011,0.000056,0.000011,0.000045,0.000011,0.000000e+00,0.000000e+00,0.000000e+00
NOT PROPN,0.000392,0.000149,0.000024,8.824373e-07,0.00000,0.000000,0.044064,0.000126,0.010960,0.000003,...,8.824373e-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.824373e-07,8.824373e-07,8.824373e-07


В целом видно, что у именованных сущностей ('РБК', 'Украина') значение в строке 'NOT PROPN' больше. У неименованных ('депутат', 'верховный') -- наоборот.

## Test

Посмотрим, какую точность модель даст на тестовой выборке.

In [50]:
test_data = words_getter(russian, 0.05, docs)

In [51]:
len(test_data)

6162

In [52]:
test_data = [(sample[0],  'PROPN' if sample[1] == 'PROPN' else 'NOT PROPN') for sample in test_data]
test_data

[('победительница', 'NOT PROPN'),
 ('конкурс', 'NOT PROPN'),
 ('«', 'NOT PROPN'),
 ('мисс', 'NOT PROPN'),
 ('саратов-2018', 'PROPN'),
 ('»', 'NOT PROPN'),
 (',', 'NOT PROPN'),
 ('также', 'NOT PROPN'),
 ('известный', 'NOT PROPN'),
 ('под', 'NOT PROPN'),
 ('название', 'NOT PROPN'),
 ('«', 'NOT PROPN'),
 ('хрустальный', 'NOT PROPN'),
 ('корона', 'NOT PROPN'),
 ('»', 'NOT PROPN'),
 (',', 'NOT PROPN'),
 ('остаться', 'NOT PROPN'),
 ('без', 'NOT PROPN'),
 ('корона', 'NOT PROPN'),
 ('на', 'NOT PROPN'),
 ('церемония', 'NOT PROPN'),
 ('награждение', 'NOT PROPN'),
 ('из-за', 'NOT PROPN'),
 ('закрытый', 'NOT PROPN'),
 ('в', 'NOT PROPN'),
 ('город', 'NOT PROPN'),
 ('завод', 'NOT PROPN'),
 ('по', 'NOT PROPN'),
 ('производство', 'NOT PROPN'),
 ('стекло', 'NOT PROPN'),
 ('и', 'NOT PROPN'),
 ('хрусталь', 'NOT PROPN'),
 ('.', 'NOT PROPN'),
 ('о', 'NOT PROPN'),
 ('это', 'NOT PROPN'),
 ('стать', 'NOT PROPN'),
 ('известно', 'NOT PROPN'),
 ('агентство', 'NOT PROPN'),
 ('«', 'NOT PROPN'),
 ('свободный', 'NOT

Если поразбираться в коде библиотеки nltk, можно понять, что вроде как надо подавать список из предложений. Где каждое предложение это список из токенов. Сделаем же в таком формате.

In [59]:
test_sequences = [[]]
i = 0
for token in test_data:
    test_sequences[i].append(token)
    if token[0] == '.':
        i += 1
        test_sequences.append([])
test_sequences.pop(-1)
test_sequences[-1]

[('кроме', 'NOT PROPN'),
 ('тот', 'NOT PROPN'),
 (',', 'NOT PROPN'),
 ('сотня', 'NOT PROPN'),
 ('тысяча', 'NOT PROPN'),
 ('человек', 'NOT PROPN'),
 ('пропасть', 'NOT PROPN'),
 ('без', 'NOT PROPN'),
 ('вести', 'NOT PROPN'),
 ('и', 'NOT PROPN'),
 ('пострадать', 'NOT PROPN'),
 ('из-за', 'NOT PROPN'),
 ('оползень', 'NOT PROPN'),
 ('и', 'NOT PROPN'),
 ('наводнение', 'NOT PROPN'),
 ('.', 'NOT PROPN')]

Теперь посчитаем acuuracy для всего тестового датасета и энтропию для каждого предложения по отдельности.

In [62]:
%%time
test_res = tagger.test(test_sequences, verbose=True)

Test: победительница/NOT PROPN конкурс/NOT PROPN «/NOT PROPN мисс/NOT PROPN саратов-2018/PROPN »/NOT PROPN ,/NOT PROPN также/NOT PROPN известный/NOT PROPN под/NOT PROPN название/NOT PROPN «/NOT PROPN хрустальный/NOT PROPN корона/NOT PROPN »/NOT PROPN ,/NOT PROPN остаться/NOT PROPN без/NOT PROPN корона/NOT PROPN на/NOT PROPN церемония/NOT PROPN награждение/NOT PROPN из-за/NOT PROPN закрытый/NOT PROPN в/NOT PROPN город/NOT PROPN завод/NOT PROPN по/NOT PROPN производство/NOT PROPN стекло/NOT PROPN и/NOT PROPN хрусталь/NOT PROPN ./NOT PROPN

Untagged: победительница конкурс « мисс саратов-2018 » , также известный под название « хрустальный корона » , остаться без корона на церемония награждение из-за закрытый в город завод по производство стекло и хрусталь .

HMM-tagged: победительница/NOT PROPN конкурс/NOT PROPN «/NOT PROPN мисс/NOT PROPN саратов-2018/PROPN »/PROPN ,/PROPN также/PROPN известный/PROPN под/PROPN название/PROPN «/PROPN хрустальный/PROPN корона/PROPN »/PROPN ,/PROPN остаться/


Untagged: с 2017 год дело о якобы российский вмешательство в американский выборы в 2016 год расследовать спецпрокурор роберт мюллер .

HMM-tagged: с/NOT PROPN 2017/NOT PROPN год/NOT PROPN дело/NOT PROPN о/NOT PROPN якобы/NOT PROPN российский/NOT PROPN вмешательство/NOT PROPN в/NOT PROPN американский/NOT PROPN выборы/NOT PROPN в/NOT PROPN 2016/NOT PROPN год/NOT PROPN расследовать/NOT PROPN спецпрокурор/NOT PROPN роберт/PROPN мюллер/PROPN ./NOT PROPN

Entropy: 0.058583451137479425

------------------------------------------------------------
Test: перед/NOT PROPN они/NOT PROPN поставить/NOT PROPN задача/NOT PROPN выяснить/NOT PROPN ,/NOT PROPN быть/NOT PROPN ли/NOT PROPN сговор/NOT PROPN между/NOT PROPN штаб/NOT PROPN трамп/PROPN и/NOT PROPN россия/PROPN ./NOT PROPN

Untagged: перед они поставить задача выяснить , быть ли сговор между штаб трамп и россия .

HMM-tagged: перед/NOT PROPN они/NOT PROPN поставить/NOT PROPN задача/NOT PROPN выяснить/NOT PROPN ,/NOT PROPN быть/NOT PROPN ли/NOT

Test: телеканал/NOT PROPN поздний/NOT PROPN объявить/NOT PROPN ,/NOT PROPN что/NOT PROPN никто/NOT PROPN из/NOT PROPN сотрудник/NOT PROPN не/NOT PROPN пострадать/NOT PROPN ./NOT PROPN

Untagged: телеканал поздний объявить , что никто из сотрудник не пострадать .

HMM-tagged: телеканал/NOT PROPN поздний/NOT PROPN объявить/NOT PROPN ,/NOT PROPN что/NOT PROPN никто/NOT PROPN из/NOT PROPN сотрудник/NOT PROPN не/NOT PROPN пострадать/NOT PROPN ./NOT PROPN

Entropy: 0.01430211701480566

------------------------------------------------------------
Test: массовый/NOT PROPN эвакуация/NOT PROPN на/NOT PROPN восточный/NOT PROPN побережье/NOT PROPN сша/PROPN в/NOT PROPN связь/NOT PROPN с/NOT PROPN ураган/NOT PROPN «/NOT PROPN флоренса/PROPN »/NOT PROPN начаться/NOT PROPN 11/NOT PROPN сентябрь/NOT PROPN ./NOT PROPN

Untagged: массовый эвакуация на восточный побережье сша в связь с ураган « флоренса » начаться 11 сентябрь .

HMM-tagged: массовый/NOT PROPN эвакуация/NOT PROPN на/NOT PROPN восточный/NO

Test: кроме/NOT PROPN тот/NOT PROPN ,/NOT PROPN от/NOT PROPN доверенный/NOT PROPN лицо/NOT PROPN врио/NOT PROPN губернатор/NOT PROPN андрей/PROPN тарасенко/PROPN поступить/NOT PROPN два/NOT PROPN жалоба/NOT PROPN на/NOT PROPN подкуп/NOT PROPN и/NOT PROPN подвоз/NOT PROPN избиратель/NOT PROPN на/NOT PROPN три/NOT PROPN участок/NOT PROPN ./NOT PROPN

Untagged: кроме тот , от доверенный лицо врио губернатор андрей тарасенко поступить два жалоба на подкуп и подвоз избиратель на три участок .

HMM-tagged: кроме/NOT PROPN тот/NOT PROPN ,/NOT PROPN от/NOT PROPN доверенный/NOT PROPN лицо/NOT PROPN врио/NOT PROPN губернатор/NOT PROPN андрей/PROPN тарасенко/PROPN поступить/NOT PROPN два/NOT PROPN жалоба/NOT PROPN на/NOT PROPN подкуп/NOT PROPN и/NOT PROPN подвоз/NOT PROPN избиратель/NOT PROPN на/NOT PROPN три/NOT PROPN участок/NOT PROPN ./NOT PROPN

Entropy: 0.15481410818858832

------------------------------------------------------------
Test: по/NOT PROPN слово/NOT PROPN представитель/NOT PROPN

Test: фотофиксация/NOT PROPN производиться/NOT PROPN до/NOT PROPN и/NOT PROPN после/NOT PROPN установка/NOT PROPN экран/NOT PROPN противометеоритный/NOT PROPN защита/NOT PROPN ./NOT PROPN

Untagged: фотофиксация производиться до и после установка экран противометеоритный защита .

HMM-tagged: фотофиксация/NOT PROPN производиться/NOT PROPN до/NOT PROPN и/NOT PROPN после/NOT PROPN установка/NOT PROPN экран/NOT PROPN противометеоритный/NOT PROPN защита/NOT PROPN ./NOT PROPN

Entropy: 0.0062713967623970746

------------------------------------------------------------
Test: по/NOT PROPN фотография/NOT PROPN никакой/NOT PROPN отверстие/NOT PROPN на/NOT PROPN корпус/NOT PROPN не/NOT PROPN обнаружить/NOT PROPN »/NOT PROPN ,/NOT PROPN —/NOT PROPN сказать/NOT PROPN он/NOT PROPN ./NOT PROPN

Untagged: по фотография никакой отверстие на корпус не обнаружить » , — сказать он .

HMM-tagged: по/NOT PROPN фотография/NOT PROPN никакой/NOT PROPN отверстие/NOT PROPN на/NOT PROPN корпус/NOT PROPN не/NOT P

Получили accuracy 85%. Довольно неплохая точность.